In [6]:
!pip install transformers datasets tweet-preprocessor ray[tune] hyperopt

     |████████████████████████████████| 1.4MB 23.2MB/s 
     |████████████████████████████████| 163kB 51.8MB/s 
     |████████████████████████████████| 23.1MB 1.2MB/s 
     |████████████████████████████████| 890kB 45.6MB/s 
     |████████████████████████████████| 2.9MB 41.3MB/s 
     |████████████████████████████████| 17.7MB 183kB/s 
     |████████████████████████████████| 245kB 59.2MB/s 
     |████████████████████████████████| 133kB 62.0MB/s 
     |████████████████████████████████| 2.9MB 45.4MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
     |████████████████████████████████| 204kB 58.0MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 1.3MB 46.0MB/s 
     |████████████████████████████████| 317kB 33.6MB/s 
     |████████████████████████████████| 61kB 10.1MB/s 
     |████████████████████████████████| 296kB 55.5MB/s 
     |████████████████████████████████| 143kB 57.8MB

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wordcloud
import preprocessor as p # tweet-preprocessor
import nltk
import re
import seaborn as sns
import torch

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.special import softmax
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm.notebook import tqdm
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# dataset_dem = pd.read_csv('/content/drive/MyDrive/democrat_tweets_v2.csv')
# dataset_gop = pd.read_csv('/content/drive/MyDrive/republican_tweets_v2.csv')

# dataset_dem["label"] = "Democrat"
# dataset_gop["label"] = "Republican"

# dataset_final = pd.concat([dataset_dem, dataset_gop])
# dataset_final.reset_index(drop=True, inplace=True)
dataset_final = pd.read_csv("/content/drive/MyDrive/Data Science Lab/2020_labled_political_tweets.csv.zip")
# dataset_final=dataset_final[(dataset_final["party"].any()=="D")]

dataset_final = dataset_final.iloc[0:10000]


for index, row in dataset_final.iterrows():
  if str(row['party']) !="D":
    if str(row["party"])!="R":
      dataset_final.drop(index, inplace=True)

dataset_final.head()
print(len(dataset_final))
  


9927


In [23]:
dataset_final.count

<bound method DataFrame.count of       Unnamed: 0                   id  ...              source  party
0              1  1212432932746473472  ...  Twitter for iPhone      D
1              2  1212390455729696768  ...  Twitter for iPhone      D
2              3  1212250054788038656  ...  Twitter for iPhone      R
3              4  1212500813593169920  ...     Twitter Web App      R
4              6  1212239323392856064  ...  Twitter for iPhone      D
...          ...                  ...  ...                 ...    ...
9995        1909  1215310037020823552  ...     Twitter Web App      R
9996        1910  1215301007829295104  ...     Twitter Web App      D
9997        1911  1215375331084963840  ...     Twitter Web App      R
9998        1912  1215309395451621376  ...     Twitter Web App      R
9999        1913  1215395091843571712  ...           TweetDeck      R

[9927 rows x 9 columns]>

In [24]:
LABEL_MAP = {
    "D": 0,
    "R": 1
}

def buildLabels(row):
    return LABEL_MAP.get(row["party"])

# def cleanTweet(row):
#   tweet = row["text"]
#   tweet = str(p.clean(tweet))
#   tweet = re.sub(r'[^\w\s]', '', tweet) # punctuation
#   tweet = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", tweet) # numbers
#   return tweet

  
dataset_final["party"] = dataset_final.apply(lambda row: buildLabels(row), axis=1)
# dataset_final["clean_text"] = dataset_final.apply(lambda row: cleanTweet(row), 
                                                  # axis=1)

In [25]:
dataset_final.head()

,Unnamed: 0,id,screen_name,user_id,time,link,text,source,party
0,1,1212432932746473472,RepLoriTrahan,1079802482640019456,2020-01-01T12:58:31-05:00,https://www.twitter.com/RepLoriTrahan/statuses...,I am proud of the work we’ve done over the pas...,Twitter for iPhone,0
1,2,1212390455729696768,RepDwightEvans,90639372,2020-01-01T10:09:44-05:00,https://www.twitter.com/RepDwightEvans/statuse...,2/ @MorethanmySLE – a cancer survivor and lupu...,Twitter for iPhone,0
2,3,1212250054788038656,RepThomasMassie,975200486,2020-01-01T00:51:50-05:00,https://www.twitter.com/RepThomasMassie/status...,"@ceQs17 Why are our people in Iraq, and how di...",Twitter for iPhone,1
3,4,1212500813593169920,SenCoryGardner,235217558,2020-01-01T17:28:15-05:00,https://www.twitter.com/SenCoryGardner/statuse...,@EnergyGOP @BLMNational @SenatorBennet @Senate...,Twitter Web App,1
4,6,1212239323392856064,RepGraceMeng,1051127714,2020-01-01T00:09:11-05:00,https://www.twitter.com/RepGraceMeng/statuses/...,"It’s 2020! As we enter a new decade, I wish ev...",Twitter for iPhone,0


In [26]:
dataset_clf = dataset_final[["text", "party"]]
dataset_clf.reset_index(drop=True, inplace=True)

In [27]:
X_train, X_val, y_train, y_val = train_test_split(dataset_clf.index.values, 
                                                  dataset_clf.party.values, 
                                                  test_size=0.20, 
                                                  random_state=42, 
                                                  stratify=dataset_clf.party.values)

dataset_clf['data_type'] = ['not_set']*dataset_final.shape[0]

dataset_clf.loc[X_train, 'data_type'] = 'train'
dataset_clf.loc[X_val, 'data_type'] = 'test'

dataset_train = dataset_clf.loc[dataset_clf.data_type == 'train']
dataset_test = dataset_clf.loc[dataset_clf.data_type == 'test']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [28]:
dataset_train.head()

,text,party,data_type
0,I am proud of the work we’ve done over the pas...,0,train
1,2/ @MorethanmySLE – a cancer survivor and lupu...,0,train
2,"@ceQs17 Why are our people in Iraq, and how di...",1,train
3,@EnergyGOP @BLMNational @SenatorBennet @Senate...,1,train
4,"It’s 2020! As we enter a new decade, I wish ev...",0,train


In [29]:
def get_dataloaders(data, batch_size):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                            do_lower_case=True)
  # tokenize train and test data so BERT can understand it
  encoded_data_train = tokenizer.batch_encode_plus(
      data[data.data_type=='train'].text.values, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      padding=True,
      max_length=64, 
      return_tensors='pt'
  )

  encoded_data_test = tokenizer.batch_encode_plus(
      data[data.data_type=='test'].text.values, 
      add_special_tokens=True, 
      return_attention_mask=True, 
      padding=True, 
      max_length=64, 
      return_tensors='pt'
  )


  # destructure out the input_ids, attention masks, and labels from tokenizer & encoder output
  input_ids_train = encoded_data_train['input_ids']
  attention_masks_train = encoded_data_train['attention_mask']
  labels_train = torch.tensor(data[data.data_type=='train'].party.values)

  input_ids_test = encoded_data_test['input_ids']
  attention_masks_test = encoded_data_test['attention_mask']
  labels_test = torch.tensor(data[data.data_type=='test'].party.values)

  train_data = TensorDataset(input_ids_train, attention_masks_train, labels_train)
  test_data = TensorDataset(input_ids_test, attention_masks_test, labels_test)

  train_dataloader = DataLoader(train_data, 
                                sampler=RandomSampler(train_data), 
                                batch_size=batch_size)

  test_dataloader = DataLoader(test_data,
                              sampler=SequentialSampler(test_data),
                              batch_size=batch_size)
  
  return train_dataloader, test_dataloader

In [30]:
def auc_score(preds, labels):
  soft_preds = softmax(preds, axis=1) # logit -> probability
  if np.shape(preds)[1] > 2: # check for multi-class
    return roc_auc_score(labels, soft_preds, multi_class='ovr')
  else:
    soft_preds = soft_preds[:,1]
    return roc_auc_score(labels, soft_preds)

def acc_score_by_class(preds, labels):
  label_dict_inverse = {v: k for k, v in LABEL_MAP.items()} 

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [31]:
def evaluate(model, dataloader, device):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []
  
  for batch in dataloader:
      
      # convert data to CUDA
      batch = tuple(b.to(device) for b in batch)
      
      inputs = {
          'input_ids':      batch[0],
          'attention_mask': batch[1],
          'labels':         batch[2],
      }

      with torch.no_grad():        
          outputs = model(**inputs) # get predictions
          
      loss = outputs[0]
      logits = outputs[1]
      loss_val_total += loss.item()

      logits = logits.detach().cpu().numpy()
      label_ids = inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)
  
  loss_val_avg = loss_val_total/len(dataloader) 
  
  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)
          
  return loss_val_avg, predictions, true_vals

In [32]:
def train_and_hyperparam_search(config,
                                model_init, # function to init a clean version of the net
                                data,       # data as Pandas array
                                cv          # rounds of cross-validation
                                ):
  losses = []
  aucs = []
  skf = StratifiedKFold(n_splits=cv, shuffle=True)
  for train_idx, test_idx in skf.split(data.text, data.party):
    model = model_init()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    print(f"Device: {device}")

    optimizer = AdamW(model.parameters(),
                    lr=config['lr'],
                    eps=config['eps'],
                    weight_decay=config['weight_decay'])
    
    data.loc[train_idx, 'data_type'] = 'train'
    data.loc[test_idx, 'data_type'] = 'test'
    
    train_dataloader, test_dataloader = get_dataloaders(data,
                                                        config['batch_size'])

    for epoch in range(1, config['epochs']+1):
      model.train() # enter training mode
      loss_train_total = 0

      for batch in train_dataloader:
          model.zero_grad()
          
          # get CUDA data
          batch = tuple(b.to(device) for b in batch)
          
          inputs = {
              'input_ids':      batch[0],
              'attention_mask': batch[1],
              'labels':         batch[2],
          }

          outputs = model(**inputs) # evaluate
          
          # for reference, we are using cross-entropy loss here,
          # as implemented in https://huggingface.co/transformers/_modules/transformers/modeling_bert.html
          loss = outputs[0]
          loss_train_total += loss.item()
          loss.backward() # do backprop

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          optimizer.step()
              
      
      loss_train_avg = loss_train_total/len(train_dataloader)    
      print(f"Training loss for epoch {epoch}: {loss_train_avg}")        
      
      val_loss, predictions, true_vals = evaluate(model, test_dataloader, device)
      auc = auc_score(predictions, true_vals)

      losses.append(val_loss)
      aucs.append(auc)

  tune.report(loss=np.mean(losses), auc=np.mean(aucs))

In [ ]:
from functools import partial

def model_init():
   return BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                        num_labels=2,
                                                        output_attentions=False,
                                                        output_hidden_states=False)   

   
config = {
    "lr": tune.choice([5e-5,3e-5,2e-5]),
    "eps": tune.loguniform(1e-10, 1e-7),
    "weight_decay": tune.loguniform(1e-10, 1e-5),
    "batch_size": tune.choice([4,8,16, 32]),
    "epochs": tune.choice([2, 3, 4])
}

scheduler = ASHAScheduler(
    metric="auc",
    mode="max",
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

reporter = CLIReporter(metric_columns=["loss", "auc", "training_iteration"])
hyperopt_search = HyperOptSearch(metric="auc", mode="max")

result = tune.run(
    partial(train_and_hyperparam_search, model_init=model_init, data=dataset_clf, cv=3),
    resources_per_trial={"cpu": 2, "gpu": 1},
    config=config,
    num_samples=8,
    scheduler=scheduler,
    search_alg=hyperopt_search,
    progress_reporter=reporter
)

2020-12-09 04:48:32,583	WARNING experiment.py:274 -- No name detected on trainable. Using DEFAULT.
2020-12-09 04:48:32,583	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 3.5/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2/2 CPUs, 1/1 GPUs, 0.0/7.18 GiB heap, 0.0/2.44 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/DEFAULT_2020-12-09_04-48-32
Number of trials: 1/8 (1 RUNNING)
+------------------+----------+-------+--------------+----------+-------------+-------+----------------+
| Trial name       | status   | loc   |   batch_size |   epochs |         eps |    lr |   weight_decay |
|------------------+----------+-------+--------------+----------+-------------+-------+----------------|
| DEFAULT_ca1f6b00 | RUNNING  |       |            8 |        4 | 7.01337e-09 | 5e-05 |    1.06147e-08 |
+------------------+----------+-------+--------------+----------+-------------+-------+----------------+




(pid=1788) 2020-12-09 04:48:42.896429: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
(pid=1788) Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
(pid=1788) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=1788) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (i

(pid=1788) Device: cuda
(pid=1788) Training loss for epoch 1: 0.6835487275451854
(pid=1788) Training loss for epoch 2: 0.6732111519949447
(pid=1788) Training loss for epoch 3: 0.595390086111743
(pid=1788) Training loss for epoch 4: 0.532020926412537


(pid=1788) Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
(pid=1788) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=1788) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=1788) Some weights of BertForSequenceClassification were 

...

(pid=8007) Device: cuda
(pid=8007) Training loss for epoch 1: 0.5418791472551904
(pid=8007) Training loss for epoch 2: 0.3133268720663832


(pid=8007) Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
(pid=8007) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=8007) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=8007) Some weights of BertForSequenceClassification were 

(pid=8007) Device: cuda
(pid=8007) Training loss for epoch 1: 0.5559328304252763
(pid=8007) Training loss for epoch 2: 0.3365363547992375


(pid=8007) Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
(pid=8007) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=8007) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=8007) Some weights of BertForSequenceClassification were 

(pid=8007) Device: cuda
(pid=8007) Training loss for epoch 1: 0.5329610580839397
